# Notebook de ejecucion de los algoritmos 

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import json
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import model_from_json
from datetime import timedelta

import statsmodels.formula.api as smf

from src.dynamic_pricing_data_loader import cargar_y_preparar_datos

from Tools.ExtractData import GetDataFromETL
from Tools.Tools4TrainRN import RedNeuronal
from Tools.Tools4ProofRN import ProofRedNeuronal
from Tools.Tools4Elasticity import Elasticity
from Tools.Tools4Clasify import K_means
from Tools.Tools4ClasSuper import ClusteringSupervisado
from Tools.Tools4ProofCluster import PredictorClusters

# Clase para extracción de los datos

In [ ]:
Data= GetDataFromETL(False)

El archivo JSON ha sido actualizado con las nuevas rutas.


## Data para ejecutar la red neuronal

In [ ]:
Df_PRN= Data.D4_Proof_NN() # Extraccion de datos para probar la Red Neuronal

## Data para ejecutar el clustering supervisado

In [ ]:
# clustering supervisado
Df_ClusSuper= Data.D4_ClusteringSuper()

## Ejecución del algoritmo de red neuronal

In [ ]:
# Dataframe de pronosticos de nuevos clientes
lista_resultados = []
Df_=Df_PRN.copy()
for row in range(len(Df_PRN)):
    InfoClient = pd.DataFrame(Df_.iloc[row]).T
    PRN=ProofRedNeuronal(InfoClient)# Clase para pronosticar nuevos con la red
    InfoClient["TARIFA DINAMICA"]= PRN.PrecioDin[0,0]
    lista_resultados.append(InfoClient)

DataTD = pd.concat(lista_resultados, ignore_index=True)

## Ejecución del algoritmo de aprendizaje supervisado

In [ ]:
# Entrena modelo ClusteringSupervisado
clustering = ClusteringSupervisado()
# Crear predictor con el modelo entrenado
predictor = PredictorClusters(Df_ClusSuper.copy())
resultados= predictor.Get_values()

In [ ]:
TD= DataTD[['NOMBRE_PASAJERO','FECHA_CORRIDA','HORA_SALIDA_CORRIDA','FECHA_OPERACION','HORA_OPERACION',
            'TIPO_PASAJERO','TARIFA DINAMICA']].copy()

In [ ]:
TP= resultados[['NOMBRE_PASAJERO','FECHA_CORRIDA','HORA_SALIDA_CORRIDA','FECHA_OPERACION','HORA_OPERACION',
            'TIPO_PASAJERO','ORIGEN_DESTINO','EMAIL','Cluster']].copy()

In [ ]:
len(TD),len(TP)

In [ ]:
len(Df_PRN),len(Df_ClusSuper)